# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [60]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [61]:
weatherDF = pd.read_csv('weather_data.csv')
weatherDF.head()

,Unnamed: 0,City ID,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,0,1489853,Tazovsky,9,RU,1658884078,78,67.4667,78.7000,49.87,7.14
1,1,897045,Solwezi,8,ZM,1658884079,61,-12.1688,26.3894,50.67,3.80
2,2,5870133,Nome,100,US,1658884079,71,64.5011,-165.4064,48.27,14.97
3,3,3466980,Caravelas,2,BR,1658883917,78,-17.7125,-39.2481,70.66,7.09
4,4,1028434,Quelimane,9,MZ,1658884079,81,-17.8786,36.8883,62.69,3.65


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [62]:
#configure gmaps
gmaps.configure(api_key=g_key)

#store lat and lng in locations and humidity in weight
locations = weatherDF[["Latitude", "Longitude"]]
weight = weatherDF["Humidity"]

In [64]:
# Convert Humidity to float and store
weather_df = weatherDF.dropna()
humidity = weatherDF["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weatherDF[["Latitude", "Longitude"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weatherDF["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

print(fig)

Figure(layout=FigureLayout(height='420px'))


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [54]:
narrowedCity = weatherDF.loc[(weatherDF["Wind Speed"] <= 10) & (weatherDF["Cloudiness"] == 0) & \
                                   (weatherDF["Max Temperature"] >= 70) & (weatherDF["Max Temperature"] <= 80)].dropna()

narrowedCity

,Unnamed: 0,City ID,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
36,36,5282297,Bethel,0,US,1658884085,64,41.3712,-73.4140,73.76,3.44
50,50,2214432,Nālūt,0,LY,1658884089,17,30.3333,10.8500,79.59,9.48
55,55,4946620,Palmer,0,US,1658884080,75,42.1584,-72.3287,70.86,3.78
69,69,360630,Cairo,0,EG,1658883947,69,30.0626,31.2497,79.56,6.91
105,105,2264557,Ponta do Sol,0,PT,1658884102,60,32.6667,-17.1000,74.12,5.86
129,129,146214,Paphos,0,CY,1658884108,41,34.7667,32.4167,74.98,4.61
252,252,2501152,Constantine,0,DZ,1658884141,38,36.3650,6.6147,76.41,1.14
376,376,1524801,Shīeli,0,KZ,1658884176,48,44.1667,66.7500,73.69,8.86
380,380,2515271,Las Palmas,0,ES,1658884177,54,28.4204,-14.0131,77.79,8.05
395,395,4983905,Albion,0,US,1658883980,77,42.2431,-84.7530,73.35,0.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [55]:
hotelDF = filtered_weatherDF

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotelDF.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City ID']}.")
    response = requests.get(base_url, params=params).json()
   
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotelDF.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: 1489853.
Closest hotel is TazHostel.
------------
Retrieving Results for Index 1: 897045.
Closest hotel is Country Lodge Solwezi.
------------
Retrieving Results for Index 3: 3466980.
Closest hotel is Pousada dos Navegantes.
------------
Retrieving Results for Index 4: 1028434.
Closest hotel is Hotel Flamingo.
------------
Retrieving Results for Index 5: 3421193.
Closest hotel is Pistut.
------------
Retrieving Results for Index 8: 3372707.
Closest hotel is Quinta da Meia Eira.
------------
Retrieving Results for Index 9: 2013279.
Missing field/result... skipping.
------------
Retrieving Results for Index 11: 3372964.
Closest hotel is Casa Das Faias.
------------
Retrieving Results for Index 12: 2126785.
Missing field/result... skipping.
------------
Retrieving Results for Index 14: 3833367.
Closest hotel is Albatross Hotel.
------------
Retrieving Results for Index 15: 5106841.
Missing field/result... skipping.
------------
Retrieving Results for Index 

Closest hotel is Gostinitsa Severyanka.
------------
Retrieving Results for Index 251: 3445350.
Closest hotel is Hotel Guest.
------------
Retrieving Results for Index 252: 2501152.
Closest hotel is Grand Hotel Cirta.
------------
Retrieving Results for Index 253: 4021858.
Closest hotel is Casa Laguna, Bed & Breakfast.
------------
Retrieving Results for Index 261: 3448596.
Closest hotel is Hotel Pousada Verde Água.
------------
Retrieving Results for Index 262: 3421719.
Closest hotel is Sleep In Narsaq.
------------
Retrieving Results for Index 267: 2644210.
Closest hotel is Premier Inn Liverpool City Centre (Moorfields) hotel.
------------
Retrieving Results for Index 269: 5110365.
Closest hotel is Chenango Valley State Park.
------------
Retrieving Results for Index 270: 2070998.
Closest hotel is Broadwater Mariner Resort.
------------
Retrieving Results for Index 276: 933995.
Closest hotel is Shanti Maurice Resort & Spa.
------------
Retrieving Results for Index 277: 3115824.
Close

Closest hotel is Panorama Motor Inn Hotel.
------------
Retrieving Results for Index 527: 1045114.
Closest hotel is Oceano.
------------
Retrieving Results for Index 529: 3351663.
Closest hotel is Hotel Praia Morena.
------------
Retrieving Results for Index 531: 2738381.
Closest hotel is Casa Maquia.
------------
Retrieving Results for Index 532: 1517637.
Closest hotel is Gostinitsa "Diyar".
------------
Retrieving Results for Index 535: 975436.
Closest hotel is Mahikeng Guest Apartments.
------------
Retrieving Results for Index 537: 497019.
Closest hotel is Guesthouse Sennoi na Zelenoi.
------------
Retrieving Results for Index 538: 3448903.
Closest hotel is Pousada Porto De Canoas.
------------
Retrieving Results for Index 540: 3860443.
Closest hotel is Austral Hotel.
------------
Retrieving Results for Index 543: 6160362.
Closest hotel is Super 8 by Wyndham Swan River MB.
------------
Retrieving Results for Index 546: 5367788.
Closest hotel is Holiday Inn Express Lompoc, an IHG Ho

In [57]:
hotelDF.head()

,Unnamed: 0,City ID,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name
0,0,1489853,Tazovsky,9,RU,1658884078,78,67.4667,78.7000,49.87,7.14,TazHostel
1,1,897045,Solwezi,8,ZM,1658884079,61,-12.1688,26.3894,50.67,3.80,Country Lodge Solwezi
3,3,3466980,Caravelas,2,BR,1658883917,78,-17.7125,-39.2481,70.66,7.09,Pousada dos Navegantes
4,4,1028434,Quelimane,9,MZ,1658884079,81,-17.8786,36.8883,62.69,3.65,Hotel Flamingo
5,5,3421193,Paamiut,19,GL,1658884079,94,61.9940,-49.6678,41.74,18.32,Pistut


In [58]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotelDF.iterrows()]
locations = hotelDF[["Latitude", "Longitude"]]

In [59]:
# Add marker layer ontop of heat map
locations = hotelDF[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)

# Display figure
# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
print(fig)

Figure(layout=FigureLayout(height='420px'))
